In [1]:
import numpy as np
import glob
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import pyemma
import msmbuilder
from msmbuilder.msm import MarkovStateModel
from msmbuilder.lumping import PCCAPlus
import sklearn
import sklearn.manifold
import scipy

/Users/rafalpwiewiora/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/rafalpwiewiora/anaconda3/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
dtrajs = [np.load(x) for x in glob.glob('dtrajs/*.npy')]

In [3]:
msm = pyemma.msm.estimate_markov_model(dtrajs, lag=100)

In [4]:
msm.nstates

66

In [5]:
pi = msm.pi

In [6]:
P = msm.P

In [20]:
np.count_nonzero(P) / (66*66)

0.21028466483011937

In [21]:
P_2 = np.linalg.matrix_power(P, 2)

In [23]:
np.count_nonzero(P_2) / (66*66)

0.5895316804407713

In [24]:
P_3 = np.linalg.matrix_power(P, 3)

In [25]:
np.count_nonzero(P_3) / (66*66)

0.8898071625344353

In [26]:
P_4 = np.linalg.matrix_power(P, 4)

In [27]:
np.count_nonzero(P_4) / (66*66)

1.0

In [28]:
fluxes_4 = np.multiply(P_4.T, pi)

In [29]:
inverse_fluxes_4_log = np.log10(1/fluxes_4)

In [30]:
# set diagonals to 0
for i in range(24):
    inverse_fluxes_4_log[i,i] = 0

In [31]:
# we want the embedding to overlap as well as possible with the macrostate embedding - load that, and the metastable
# sets from the HMM - we will generate a microstate embedding and calculate the distances from microstates belonging to 
# macrostate X to that macrostate in the HMM embedding - make 1000 embeddings, pick the top 50 with lowest sums of
# distances

In [33]:
hmm_embedding = np.load('/Users/rafalpwiewiora/repos/MSM/11708_11710_SET8_ligands/dih/SAM/hmms/results/inverse_flux_150ns_transmatrix_log10_embedding/save_data/17.npy')

In [34]:
meta_sets = np.load('/Users/rafalpwiewiora/repos/MSM/11708_11710_SET8_ligands/dih/SAM/hmms/results/12/metastable_sets.npy')

In [44]:
meta_sets

array([array([ 2,  5,  7, 11, 21, 23, 25, 34, 54, 55, 60, 65]),
       array([13, 19]), array([ 0, 18, 31]), array([46]),
       array([ 1,  6, 28, 43, 48]),
       array([ 4, 16, 20, 30, 32, 33, 35, 39, 50, 58, 59, 62, 64]),
       array([44, 49, 51]),
       array([ 3,  9, 12, 15, 17, 22, 24, 27, 36, 37, 40, 42, 45, 47, 53, 56, 61,
       63, 66]),
       array([26, 29]), array([ 8, 10, 14, 38, 41, 52, 57])], dtype=object)

In [43]:
msm.connected_sets[0]

array([ 0,  1,  2,  3,  4,  6,  7,  8, 11, 13, 14, 15, 16, 18, 19, 21, 23,
       25, 26, 27, 29, 30, 33, 36, 37, 38, 41, 42, 43, 44, 49, 50, 51, 54,
       55, 57, 58, 59, 61, 62, 63, 65, 66, 67, 68, 72, 73, 74, 76, 79, 80,
       81, 82, 83, 84, 85, 86, 87, 89, 92, 93, 94, 95, 96, 97, 98])

In [ ]:
# make a microstate: macrostate dictionary - first translate 

In [46]:
hmm_micro_to_micro_translate = {0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 6: 5,
 7: 6,
 8: 7,
 11: 8,
 13: 9,
 14: 10,
 15: 11,
 16: 12,
 18: 13,
 19: 14,
 21: 15,
 23: 16,
 25: 17,
 26: 18,
 27: 19,
 29: 20,
 30: 21,
 33: 22,
 36: 23,
 37: 24,
 38: 25,
 41: 26,
 42: 27,
 43: 28,
 44: 29,
 45: 30,
 49: 31,
 50: 32,
 51: 33,
 54: 34,
 55: 35,
 57: 36,
 58: 37,
 59: 38,
 61: 39,
 62: 40,
 63: 41,
 65: 42,
 66: 43,
 67: 44,
 68: 45,
 72: 46,
 73: 47,
 74: 48,
 76: 49,
 79: 50,
 80: 51,
 81: 52,
 82: 53,
 83: 54,
 84: 55,
 85: 56,
 86: 57,
 87: 58,
 89: 59,
 92: 60,
 93: 61,
 94: 62,
 95: 63,
 96: 64,
 97: 65,
 98: 66}

In [48]:
micro_macro_translate = dict()

for msm_microstate_index, dtrajs_microstate_index in enumerate(msm.connected_sets[0]):
    hmm_microstate = hmm_micro_to_micro_translate[dtrajs_microstate_index]
    for macro_index, macrostate in enumerate(meta_sets):
        if hmm_microstate in macrostate:
            macrostate_index = macro_index
            break
    micro_macro_translate[msm_microstate_index] = macrostate_index

In [50]:
# make a dictionary of microstate: macrostate it belongs to coordinates

In [51]:
micro_coordinates = dict()

for msm_microstate_index, dtrajs_microstate_index in enumerate(msm.connected_sets[0]):
    hmm_microstate = hmm_micro_to_micro_translate[dtrajs_microstate_index]
    for macro_index, macrostate in enumerate(meta_sets):
        if hmm_microstate in macrostate:
            macrostate_index = macro_index
            break
    micro_coordinates[msm_microstate_index] = hmm_embedding[macrostate_index]

In [52]:
embeddings = []

for j in range(1000):
    mds = sklearn.manifold.MDS(dissimilarity='precomputed')
    mds_embed = mds.fit_transform(inverse_fluxes_4_log)
    distance_sum = 0
    for microstate_index, microstate in enumerate(mds_embed):
        distance = scipy.spatial.distance.pdist([microstate, micro_coordinates[microstate_index]])
        distance_sum += distance[0]
    embeddings.append([mds_embed, distance_sum])    

In [53]:
# sort by distance
sorted_embeddings = sorted(embeddings, key=lambda x: x[1])

In [54]:
for j in range(50):
    mds_embed = sorted_embeddings[j][0]
    np.save('inverse_flux_200ns_transmatrix_log10_embedding/save_data/%d.npy' % j, mds_embed)
    with open('inverse_flux_200ns_transmatrix_log10_embedding/%d.txt' % j, 'w') as f:
        f.write(str(mds_embed))
    f, ax = plt.subplots(dpi=300)
    ax.scatter(mds_embed[:,0], mds_embed[:,1], c='black')
    for i in range(66):
        ax.annotate(i, (mds_embed[i,0], mds_embed[i,1]))
    plt.savefig('inverse_flux_200ns_transmatrix_log10_embedding/%d.png' % j, dpi=300)
    plt.close()

In [55]:
for j in range(50):
    mds_embed = sorted_embeddings[j][0]
    f, ax = plt.subplots(dpi=300)
    ax.scatter(mds_embed[:,0], mds_embed[:,1], c='black')
    for i in range(66):
        ax.annotate(micro_macro_translate[i], (mds_embed[i,0], mds_embed[i,1]))
    plt.savefig('inverse_flux_200ns_transmatrix_log10_embedding/%d_macrolabels.png' % j, dpi=300)
    plt.close()